In [3]:
import json
api_token = {"username":"iobananaoi","key":"0cbdda23beeb18cbb57e9e88bf26bfd1"}

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 /root/.kaggle/kaggle.json

In [49]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import zipfile
import cv2
import os
import kaggle
from torchvision.transforms import ToTensor

In [47]:
batch_size = 64
lr = 1e-3
epochs = 10
device = "cuda" if torch.cuda.is_available else "cpu"
device

'cuda'

# Data processing

In [6]:
!kaggle datasets download -d balraj98/summer2winter-yosemite

 97% 123M/126M [00:01<00:00, 102MB/s]
100% 126M/126M [00:01<00:00, 110MB/s]


In [13]:
f = zipfile.ZipFile("/content/summer2winter-yosemite.zip")
f.extractall("/content/imgs")

In [50]:
class HrLrImages(Dataset):
    def __init__(self, src = "/content/imgs/trainA/", transform=None):
        self.src = src
        self.transform = transform
        
        imgs = os.listdir(src)
        if not os.path.isdir(src+"low_res"):
            os.mkdir(src+"low_res")

        for i, img in enumerate(imgs):
            if img[-4:] == ".jpg":
                new_name = str(i) + ".jpg"
                os.rename(src+img, src+new_name)
                img_hr = cv2.imread(src+new_name)
                cv2.imwrite(src+"low_res/"+new_name, img_hr, [int(cv2.IMWRITE_JPEG_QUALITY), 10])
            

    def __len__(self):
        return len(os.listdir(self.src))

    def __getitem__(self, idx):
        name = str(idx)+".jpg"
        img_hr = cv2.imread(self.src+name)
        img_lr = cv2.imread(self.src+"low_res/"+ name)

        if self.transform is not None:
            img_hr = self.transform(img_hr)
            img_lr = self.transform(img_lr)

        return img_hr, img_lr

In [52]:
dataset = HrLrImages(transform=ToTensor())

In [53]:
dataloader = DataLoader(dataset, batch_size, shuffle=True)

In [54]:
dataset[0][0].shape

torch.Size([3, 256, 256])

# Model structure

In [71]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()

        # Encoder part
        
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=2, dilation=1),
            nn.BatchNorm2d(32),
            nn.PReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=2, dilation=1),
            nn.BatchNorm2d(64),
            nn.PReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=2, dilation=1),
            nn.BatchNorm2d(64),
            nn.PReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(69696, 20096),
            nn.ReLU(),
            nn.Linear(20096, 5024),
            nn.ReLU(),
            nn.Linear(5024, 1256)
        )

        # Decoder part: the output should be a 256x256 image

        self.decoder = nn.Sequential(
            torch.nn.Linear(1256, 5024),
            torch.nn.ReLU(),
            torch.nn.Linear(5024, 20096),
            torch.nn.ReLU(),
            torch.nn.Linear(20096, 256*256),
            torch.nn.Sigmoid()
        )

    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Training part

In [72]:
def train(model, dataloader, optimizer, loss_fn):
    # for epoch in range(epochs):
    #print(f"======== Epoch {epoch+1} ========")
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        print(pred)
        break

In [73]:
model = AE()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
train(model, dataloader, optimizer, loss_fn)